In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [2]:
df = pd.read_csv("city_day.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'city_day.csv'

In [3]:
import zipfile

with zipfile.ZipFile('/content/city_day.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [4]:
df = pd.read_csv("/content/city_day.csv")
display(df.head())

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [5]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

In [6]:
data = df[['Date', 'PM2.5']].dropna()
data = data.set_index('Date')


In [7]:
train = data.iloc[:-30]
test = data.iloc[-30:]

In [9]:
%pip install pmdarima

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.2 MB/s eta 0:00:00


In [22]:
try:
    from pmdarima import auto_arima


    arima_model = auto_arima(
        train['PM2.5'],
        seasonal=False,
        stepwise=True,
        suppress_warnings=True,
        error_action='ignore'
    )


    arima_pred = arima_model.predict(n_periods=len(test))


    rmse_arima = np.sqrt(mean_squared_error(test['PM2.5'], arima_pred))
    mae_arima = mean_absolute_error(test['PM2.5'], arima_pred)

    print("ARIMA RMSE:", rmse_arima)
    print("ARIMA MAE:", mae_arima)

except Exception as e:
    print("⚠️ ARIMA model failed due to:", e)
    rmse_arima = np.nan
    mae_arima = np.nan


⚠️ ARIMA model failed due to: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject


In [10]:
prophet_df = train.reset_index().rename(columns={'Date': 'ds', 'PM2.5': 'y'})
prophet_model = Prophet()
prophet_model.fit(prophet_df)

future = test.reset_index().rename(columns={'Date': 'ds'})
forecast = prophet_model.predict(future)
prophet_pred = forecast['yhat'].values

rmse_prophet = np.sqrt(mean_squared_error(test['PM2.5'], prophet_pred))
mae_prophet = mean_absolute_error(test['PM2.5'], prophet_pred)

print("Prophet RMSE:", rmse_prophet)
print("Prophet MAE:", mae_prophet)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp00pij382/emszka2b.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp00pij382/djk91m9_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=57330', 'data', 'file=/tmp/tmp00pij382/emszka2b.json', 'init=/tmp/tmp00pij382/djk91m9_.json', 'output', 'file=/tmp/tmp00pij382/prophet_modely3_8e5b5/prophet_model-20251020125507.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:55:07 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:55:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet RMSE: 18.99726812898314
Prophet MAE: 14.036913956849043


In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train = scaler.fit_transform(train[['PM2.5']])

In [14]:
X, y = [], []
for i in range(10, len(scaled_train)):
    X.append(scaled_train[i-10:i])
    y.append(scaled_train[i])
X, y = np.array(X), np.array(y)

lstm_model = Sequential([
    LSTM(64, activation='relu', input_shape=(X.shape[1], 1)),Dense(1)])
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X, y, epochs=25, batch_size=8, verbose=0)

In [20]:
total_data = pd.concat((train, test))
inputs = scaler.transform(total_data[['PM2.5']])
X_test = []
for i in range(len(train), len(total_data)):
    X_test.append(inputs[i-10:i])
X_test = np.array(X_test)

lstm_pred_scaled = lstm_model.predict(X_test)
lstm_pred = scaler.inverse_transform(lstm_pred_scaled)

rmse_lstm = np.sqrt(mean_squared_error(test['PM2.5'], lstm_pred))
mae_lstm = mean_absolute_error(test['PM2.5'], lstm_pred)

print("LSTM RMSE:", rmse_lstm)
print("LSTM MAE:", mae_lstm)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
LSTM RMSE: 19.72956968774725
LSTM MAE: 16.92564275868734


In [23]:
results = pd.DataFrame({
    'Model': ['ARIMA', 'Prophet', 'LSTM'],
    'RMSE': [rmse_arima, rmse_prophet, rmse_lstm],
    'MAE': [mae_arima, mae_prophet, mae_lstm]
})

print("\nModel Performance Comparison:")
print(results)

best_model_name = results.loc[results['RMSE'].idxmin(), 'Model']
print(f"\n✅ Best Model: {best_model_name}")


Model Performance Comparison:
     Model       RMSE        MAE
0    ARIMA        NaN        NaN
1  Prophet  18.997268  14.036914
2     LSTM  19.729570  16.925643

✅ Best Model: Prophet


In [24]:
if best_model_name == 'ARIMA':
    joblib.dump(arima_model, 'best_pollution_model.pkl')
elif best_model_name == 'Prophet':
    joblib.dump(prophet_model, 'best_pollution_model.pkl')
else:
    lstm_model.save('best_pollution_model_lstm.h5')

print("\n✅ Best model saved successfully for inference.")


✅ Best model saved successfully for inference.
